In [1]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [2]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/8章'
# os.chdir(path)
# ! pip install japanize-matplotlib

# ８章 構築した機械学習モデルで新規データを予測する１０本ノック



### ノック７１：フォルダ生成をしてデータ読み込みの準備をしよう

In [4]:
import os
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "1_output")
master_dir = os.path.join(data_dir, "99_master")
model_dir = "models"
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

##### ＊＊必ずデータやモデルの配置をおこなってください。＊＊

### ノック７２：予測したい新規データを読み込もう

In [5]:
import pandas as pd
m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))

In [6]:
tg_ym = "202003"
target_file = "tbl_order_" + tg_ym + ".csv"
target_data = pd.read_csv(os.path.join(input_dir, target_file))

import datetime
max_date = pd.to_datetime(target_data["order_accept_date"]).max()
min_date = pd.to_datetime(target_data["order_accept_date"]).min()
max_str_date = max_date.strftime("%Y%m")
min_str_date = min_date.strftime("%Y%m")
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

日付が一致しました


### ノック７３：新規データを店舗別で集計しよう

### ノック７４：新規データのカテゴリカル変数対応をしよう

### ノック７５：モデルに投入する直前の形式に整えよう

### ノック７６：モデルファイルを読み込んでみよう

### ノック７７：新規データの予測をしてみよう

### ノック７８：予測結果のヒートマップを作成してみよう

### ノック７９：実績データを作成しよう

### ノック８０：現場に向けたレポートを作成し出力しよう